# CNN으로 패션 아이템 구분하기

Convolutional Neural Network (CNN) 을 이용하여 패션아이템 구분 성능을 높여보겠다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS     = 40
BATCH_SIZE = 64

## 데이터셋 불러오기

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))       # 정규화
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))       # 정규화
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

## 뉴럴넷으로 Fashion MNIST 학습하기

**학습 모델 정의**

* `nn.Conv2d` 모듈: 첫 두 파라미터는 입력 채널 수 (in_channels)와 출력 채널 수 (out_channels)이다. Fashion MNIST 데이터셋은 흑백이미지이므로 색상채널이 1개뿐이다.
* 첫 컨벌루션 층에서는 10개의 특징맵을 생성하고, 두 번째  컨벌루션 층에서는 10개의 특징맵을 받아 20개의 특징맵을 생성한다.
* 각 컨벌루션 층에서 커널의 크기는 kernel_size로 지정할 수 있으며, 숫자를 하나만 지정하면 정사각형 커널로 간주한다. 즉 kernel_size=5는 kernel_size=(5,5)를 의미한다.
* 컨볼루션 결과로 나온 출력값에는 드롭아웃을 해준다. 드롭아웃 함수 `F.dropout` 대신 `nn.Dropout2d` 모듈을 사용하여 드롭아웃 인스턴스를 만든다.
* 컨볼루션 층과 드롭아웃을 거친 이미지는 일반 신경망을 거친다. 첫 번째 신경망 층에서는 앞 층의 출력 크기인 320을 입력 크기로 하고, 출력 크기를 50으로 한다. 두 번째 신경망 층에서는 입력 크기를 50으로 하고, 출력 크기는 분류할 클래스 개수인 10으로 설정한다.
* $이미지: 28*28*1 -> 첫 번째 합성곱: 5*5*10 (valid padding) ->  첫 번째 합성곱 후: 24*24*10 -> 첫 번째 풀링: 2*2 -> 첫 번째 풀링 후: 12*12*10 ->  두 번째 합성곱: 5*5*20 (valid padding) ->  두 번째 합성곱 후: 8*8*20 -> 두 번째 풀링: 2*2 -> 두 번째 풀링 후: 4*4*20 (=320) 
-> (Flattened)$

<br>

**입력에서 출력까지 데이터가 지나갈 길 생성**

* 입력 x를 받아 첫 번째 컨볼루션 층을 거치게 한 후 풀링 함수 `F.max_pool2d`를 거치게 한다. `F.max_pool2d` 함수의 두 번째 입력은 커널 크기이다. 풀링 연산은 드롭아웃과 같이 학습 파라미터가 없으므로 개인 취향에 따라 풀링 함수 `F.max_pool2d`를 사용하거나 nn.MaxPool2d 모듈을 사용해도 좋다. 컨볼루션과 최대 풀링을 통과한 x는 활성화함수 `F.relu`를 거친다.
* 컨볼루션 층 2개를 거친 특징맵이 된 x는 2차원 구조이다. 특징맵 이후의 출력을 하는 일반 인공 신경망은 1차원 입력을 받는다. 2차원의 특징맵을 바로 입력으로 넣을 수 없으므로 1차원으로 펴주도록 한다 (Flattened). `view` 메서드에 들어가는 첫 번째 입력 -1은 '남은 차원 모두'를 의미하며, 320은 X가 가진 원소 개수를 의미한다.
* 앞서 추출한 특징들을 입력으로 받아 분류하는 신경망 층을 구성해 보면 첫 번째 층에서는 ReLU 활성화 함수를 거치도록 하고 과적합을 방지하기 위해 드롭아웃을 사용한다.
* 마지막 층은 10개의 출력값 (0부터 9까지의 레이블을 갖는)을 가지는 신경망이다.

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)              
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## 하이퍼파라미터 

* `to()` 함수는 모델의 파라미터들을 지정한 곳으로 보내는 역할을 한다. 일반적으로 CPU 1개만 사용할 경우 필요는 없지만, GPU를 사용하고자 하는 경우 `to("cuda")`로 지정하여 GPU로 보내야 한다. 지정하지 않을 경우 계속 CPU에 남아 있게 되며 빠른 훈련의 이점을 누리실 수 없다.
* 최적화 알고리즘으로 파이토치에 내장되어 있는 `optim.SGD`를 사용하겠다.

In [8]:
model     = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

## 학습하기

In [9]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

## 테스트하기

* 평가를 위해서는 교차 엔트로피를 거칠 때 `reduction='sum'`을 지정해주어 미니배치의 평균 대신 합을 받아와야 한다.

In [10]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 코드 돌려보기

자, 이제 모든 준비가 끝났다. 코드를 돌려서 실제로 학습이 되는지 확인해보자!

In [11]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306020
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.444399
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.732238
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.632071
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.702729
[1] Test Loss: 0.2164, Accuracy: 93.78%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.327125
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.350220
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.467204
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.519017
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.177037
[2] Test Loss: 0.1316, Accuracy: 95.96%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.507785
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.602472
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.354356
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.204434
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.406728
[3] Test Loss: 0.1026, Accuracy: 96.75%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.480839
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.474030
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.531560


Train Epoch: 29 [38400/60000 (64%)]	Loss: 0.418544
Train Epoch: 29 [51200/60000 (85%)]	Loss: 0.049143
[29] Test Loss: 0.0346, Accuracy: 98.96%
Train Epoch: 30 [0/60000 (0%)]	Loss: 0.257067
Train Epoch: 30 [12800/60000 (21%)]	Loss: 0.125538
Train Epoch: 30 [25600/60000 (43%)]	Loss: 0.056162
Train Epoch: 30 [38400/60000 (64%)]	Loss: 0.119025
Train Epoch: 30 [51200/60000 (85%)]	Loss: 0.089635
[30] Test Loss: 0.0329, Accuracy: 98.89%
Train Epoch: 31 [0/60000 (0%)]	Loss: 0.072931
Train Epoch: 31 [12800/60000 (21%)]	Loss: 0.107166
Train Epoch: 31 [25600/60000 (43%)]	Loss: 0.237537
Train Epoch: 31 [38400/60000 (64%)]	Loss: 0.179326
Train Epoch: 31 [51200/60000 (85%)]	Loss: 0.297881
[31] Test Loss: 0.0339, Accuracy: 98.86%
Train Epoch: 32 [0/60000 (0%)]	Loss: 0.069051
Train Epoch: 32 [12800/60000 (21%)]	Loss: 0.280302
Train Epoch: 32 [25600/60000 (43%)]	Loss: 0.033259
Train Epoch: 32 [38400/60000 (64%)]	Loss: 0.098652
Train Epoch: 32 [51200/60000 (85%)]	Loss: 0.174550
[32] Test Loss: 0.0339, A

**수행 결과**

* PyTorch 하에서 컨볼루션 두 번 (커널 10, 20개 사용), 드롭아웃 사용: 테스트 데이터에서의 정확도 98.91%
* Pytorch 하에서 DNN + 드롭아웃: 테스트 데이터에서의 정확도 97.15%
* Pytorch 하에서 DNN: 테스트 데이터에서의 정확도 86.7%

* 비교: 텐서플로 하에서 VGG-7 사용 (컨볼루션 네 번 (커널 32, 64, 128, 256개 사용), 드롭아웃 사용): 테스트 데이터에서의 정확도 91.6%

(생각해 볼 문제) 1. 정확도를 높이기 위해 적절한 모형은 무엇인가?
                 2. 데이터셋에 따라 과도한 학습모형은 오히려 독이 된다!